In [87]:
import numpy as np
import matplotlib
import cv2

### Functions to Display images and plot some points over an image

In [88]:
def display_image(window_name, img):
    """Displays image with given window name

    :param window_name: name of the window
    :param img: image object to display
    """
    cv2.imshow(window_name, img.astype(np.uint8))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def draw_landmarks(window_name, img, landmarks):
    """Draws the landmark points on the image

    :param window_name: name of the window
    :param img: image object to display
    :param landmarks: landmark points [N x 2]
    """
    img_ = np.copy(img)
    for landmark in landmarks:
        cv2.circle(img_, (int(landmark[0]), int(landmark[1])), 2, (0, 255, 0), 2)
    display_image(window_name, img_)

### Functions to compute the 2D Euclidean Distance Transform on a Binary Edges Image

We first define a function to compute the distance transform as a single dimension case for each image column

Then, the above result for all the columns is passed through the 1D distance transform again, this time row-wise to complete the evaluation for 

In [89]:
def l2_distance_transform_1D(row_vec):
    """Compute the L2 distance transform for a n-element vector
    :param row_vec: numpy array containing the Edge intensity values
    """
    l = len(row_vec)
    l2_dist_tf = np.zeros(l)
    v = np.zeros(l, np.int32)

    z = np.empty(l + 1)
    z[0] = -np.inf
    z[1] = np.inf

    k = 0
    q = 1
    while q < l:
        s = 0.5 * ((row_vec[q] + q ** 2) - (row_vec[v[k]] + v[k] ** 2)) / (q - v[k])
        if s <= z[k]:
            k = k - 1
            continue
        else:
            k = k + 1
            v[k] = q
            z[k] = s
            z[k + 1] = np.inf
            q = q + 1

    k = 0
    for q in range(l):
        while z[k + 1] < q:
            k = k + 1
        l2_dist_tf[q] = (q - v[k]) ** 2 + row_vec[v[k]]

    return l2_dist_tf


def l2_distance_transform_2D(edge_mat):
    """Compute the L2 distance transform for a [m x n] matrix
    :param edge_mat: numpy array containing the Edge intensity values
    """
    dist_column_wise = np.zeros_like(edge_mat, np.uint32)
    # Compute Distance Transform for each column
    for col in range(edge_mat.shape[1]):
        dist_column_wise[:, col] = l2_distance_transform_1D(edge_mat[:, col])
    
    # Compute Distance Transform for each row
    l2_dist_tf = np.zeros_like(edge_mat, np.uint32)
    for row in range(edge_mat.shape[0]):
        l2_dist_tf[row] = l2_distance_transform_1D(dist_column_wise[row])

    l2_dist_tf = l2_dist_tf * 255 / np.max(l2_dist_tf)
    return l2_dist_tf

#### Some Functions defined to complete the task

- Function to compute the gradient of an image:
  - It uses the central difference to compute the gradient along each axis of the image
- Computing the nearest points on the object contour
  - Here, we use the approximation formula (from the slides) for computing the derivative of the Maximum Likelihood with respect to the Affine Transformation matrix
  - This Derivative evaluated at each landmark location is used as the search direction for the new corresponding landmark locations
- Computing the Affine Transform
  - We have the source landmark points and compute the target landmark points using the above method
  - We have each point (x, y) in the source associated with a point (x', y') in the target, which are related as follows
  $$
    \left(\begin{array}{cc} 
    x'\\
    y'
    \end{array}\right) = 
    \left(\begin{array}{cc} 
    m_1 & m_2\\
    m_3 & m_4
    \end{array}\right)
    \left(\begin{array}{cc} 
    x\\ 
    y
    \end{array}\right)  + \left(\begin{array}{cc} 
    t_1\\
    t_2
    \end{array}\right)
  $$

  - We rewrite the above equation as follows to come up with a system of linear equations in the unknowns we have
  $$
    \left(\begin{array}{cc} 
    x'\\
    y'
    \end{array}\right) = 
    \left(\begin{array}{cc} 
    x & y & 1 & 0 & 0 & 0\\
    0 & 0 & 0 & x & y & 1
    \end{array}\right)
    \left(\begin{array}{cc} 
    m_1\\ 
    m_2\\
    t_1\\
    m_3\\
    m_4\\
    t_2
    \end{array}\right)
  $$
  - We can concatenate the above matrices for each of the landmark points to have a system of equations $Ax\ =\ b$
  - Now that we have a system of equations, we can use the pseudo inverse of $A$ to solve for $x$

In [102]:
def compute_gradient(image):
    """Compute the gradients of an image using central differences
    :param image: numpy array of the input image to be processed
    """
    kernel = np.array([[0, 0, 0], [-1, 0, 1], [0, 0, 0]])
    grad_x = cv2.filter2D(image, -1, kernel)
    grad_y = cv2.filter2D(image, -1, kernel.T)
    return grad_x, grad_y
    
def compute_nearest_pts(landmarks, DistTF, grad_x, grad_y, grad_mag):
    """Approximate the nearest points to the starting landmark positions using the distance transform and gradient information available
    :param landmarks: numpy array [n x 2] of the input landmark locations
    :param DistTF: distance transform computed over the binary edges in the input image
    :param grad_x: gradient of the distance transform in the x direction (cols)
    :param grad_y: gradient of the distance transform in the y direction (rows)
    :param grad_mag: gradient magnitude of the distance transform
    """
    # Approximate the nearest integer pixel location to the input landmarks
    y = landmarks[:, 1].astype(int)
    x = landmarks[:, 0].astype(int)

    # Approximate the nearest points on the edges in the image by moving along the normal direction to the input landmarks
    nearest_pts = landmarks - np.multiply(np.divide(DistTF[y, x], grad_mag[y, x], where=grad_mag[y, x]!=0).reshape(-1, 1), np.dstack((grad_x[y, x], grad_y[y, x]))[0])
    return nearest_pts

def compute_affine_tf(source_pts, target_pts):
    """Compute the 2D Affine transform between the source and target set of landmarks
    :param source_pts: numpy array [n x 2] of the source landmark points
    :param target_pts: numpy array [n x 2] of the target landmark points
    """
    # Convert source points to homogenous coordinates for ease of computation later
    source_pts_hom = np.hstack((source_pts, np.ones((len(source_pts), 1))))

    # 2D Affine Transformation Matrix
    # [[m1, m2, t1]
    #  [m3, m4, t2]
    #  [ 0,  0, 1 ]]

    # Order of unknowns [m1, m2, t1, m3, m4, t2]

    # Compute the transition matrix using the source landmark points
    A = np.array([np.kron(np.eye(2), pt) for pt in source_pts_hom]).reshape(-1, 6)
    b = target_pts.reshape(-1, 1)

    # Compute unknowns using the built-in pseudo-inverse method from numpy linalg pack
    x = np.linalg.pinv(A.astype(float)) @ b

    # reshape the output vector into the transformation matrix format
    T = np.eye(3)
    T[:2, :] = x.reshape(2, 3)
    return T
    



In [98]:
# Read Image
img = cv2.imread("data/hand.jpg")

# Detect Edges in the image using Canny Edge Detector and convert the result to a binary image {0, 255}
canny_edges = cv2.Canny(img, 75, 90)
binary_edges = np.where(canny_edges == 255, 0, img.shape[0] ** 2 + img.shape[1] ** 2)

# Compute the L2 Distance transform over the Binary edge Image
l2_dist_tf = l2_distance_transform_2D(binary_edges)

# Compute gradients of the distance transform function output
grad_x, grad_y = compute_gradient(l2_dist_tf)
grad_mag = np.sqrt(np.square(grad_x) + np.square(grad_y))

# Read and Process the landmarks given
landmarks_ip = np.loadtxt("data/hand_landmarks.txt", tuple, delimiter=',', converters={0: lambda x: int(str(x).strip('b\'(')), 1: lambda y: int(str(y).strip('b\')'))})
draw_landmarks('Input Shape Model', img, landmarks_ip)


### Here we use all the functionalities defined in the cells above to run the ICP algorithm and compute the final transformation from given landmark coordiantes to the contour coordinates in the image.

- We start the algorithm assuming an Identity transformation as the given landmarks lie within the bounds of the image already.
- We keep a track of the previous and the new coordinates of the landmarks and terminate the algorithm if the absolute (l2 norm) change between them is below a threshold 

In [103]:
threshold = 1
change = np.inf
T = np.eye(3)

landmarks_old = np.copy(landmarks_ip)
landmarks_new = np.zeros_like(landmarks_old)
while change > threshold:
    landmarks_new = compute_nearest_pts(landmarks_old, l2_dist_tf, grad_x, grad_y, grad_mag)
    T = T @ compute_affine_tf(landmarks_old, landmarks_new)
    change = np.sum(np.linalg.norm((landmarks_new - landmarks_old).astype(float), 2, 1))
    landmarks_old = landmarks_new

landmarks_final = T[:2, :2] @ landmarks_ip.T + T[:2, -1].reshape(-1, 1)
draw_landmarks('Final Shape Model: After ICP', img, landmarks_final.T)

- It can be seen from the result that the algorithm fails too converge to the actual boundary of the hand on the right side due to the presence of shadow which leads to misleading edge information
- The algorithm is also confused in highly confined spaces like the space between the fingers as it has edges closeby in multiple directions